In [ ]:
%load_ext autoreload
%autoreload 2

%cd '..'

In [ ]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(
            f"data/logs/abortion_filter_comments.log"
        ),
        logging.StreamHandler(stream=sys.stdout),
    ],
)


In [ ]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

from load.utils import (
    load_df_from_parquet,
)
from preprocessing.utils import (
    save_event_comments,
    load_event_comments,
    build_vocab,
    save_event_vocab,
)

from preprocessing.constants import (
    MIN_OCCURENCE_FOR_VOCAB,
)

from events.abortion_constants import (
    ABORTION_EVENTS_INFO,
    ABORTION_KEYWORDS,
)
from events.utils import get_event_regex

In [ ]:
YEARS = [2015, 2016, 2017, 2018]

OVERALL_EVENT_NAME = "abortion"
EVENT_NAMES = list(ABORTION_EVENTS_INFO.keys())

## Filter event comments

In [ ]:
logging.info("Load partisan comments")
partisan_comments_list = []
for year in YEARS:
    logging.info(f"Loading user partisan comments from {year}...")
    partisan_comments_year = load_df_from_parquet(
        file_name=f"user_partisan_comments_{year}.parquet",
    )

    partisan_comments_year = partisan_comments_year[
        partisan_comments_year["party"].isin(["dem", "rep"])
    ]

    partisan_comments_list.append(partisan_comments_year)

partisan_comments = pd.concat(partisan_comments_list, ignore_index=True)


In [ ]:
ABORTION_REGEX = get_event_regex(ABORTION_KEYWORDS, [], "or")

In [ ]:
logging.info("Filtering event data based on keywords...")
event_comments = partisan_comments[
    partisan_comments["tokens"].str.contains(
        ABORTION_REGEX,
        regex=True,
    )
].copy()
logging.info("finished keyword filtering")


In [ ]:
logging.info(event_comments.shape)
logging.info(event_comments.dtypes)

In [ ]:
save_event_comments(event_comments, OVERALL_EVENT_NAME)

In [ ]:
del partisan_comments

In [ ]:
logging.info(f"Nr of event comments: {len(event_comments)}")

## Build event vocabulary

In [ ]:
# Read event data
# events_comments = load_event_comments(
#     theme="abortion",
#     event_name=OVERALL_EVENT_NAME,
# )

event_vocab = build_vocab(
    event_comments["tokens"],
    ngram_range=(1, 2),
    min_df=MIN_OCCURENCE_FOR_VOCAB,
)


In [ ]:
logging.info("Vocabulary length")
logging.info(len(event_vocab))

In [ ]:
logging.info("Saving event vocab")
save_event_vocab(event_vocab, OVERALL_EVENT_NAME)